# ERD/ERS Analysis Pipeline

This notebook provides a reusable pipeline to process EEG data for a simple ERD/ERS analysis.
It:
- Loads pre-processed EEG data for a participant.
- Creates and merges events from annotations.
- Segments the data into epochs (for pre-set conditions).
- Computes time–frequency representations (TFRs) with Morlet wavelets.
- Processes the TFR output to compute ERD/ERS values using baseline normalization,
  including averaging across regions of interest (ROIs).
- Exports and/or plots the final results.

You can change key parameters (e.g., file paths, channel selections, epoch timing, etc.)
in the **Parameters** section below.



In [ ]:
# Import Packages
import os
import numpy as np
import pandas as pd
import mne
from mne.time_frequency import tfr_morlet
import matplotlib.pyplot as plt

## Parameters

Set key parameters for the analysis including file paths, channels, epoch settings, ...:

In [ ]:
# Parameters (user adjustable)
params = {
    'participant_files': {
        # Change these paths to point to your participant .fif files.
        'P1': r'C:\Users\...',
        'P2': r'C:\Users\...',
        # Add more participant file paths as needed
    },
    'event_ids': {
        # Map annotation labels to numeric codes, change the following to your event IDs:
        'Stimulus/S  3': 3,
        'Stimulus/S  4': 4,
        'Stimulus/S 11': 11,
        'Stimulus/S 12': 12,
        'Stimulus/S 13': 13,
        'Stimulus/S 30': 30
        # Add more stimuli as needed
    },
    'merge_events': {
        # Define groups to merge: for instance, merge 3 and 4 into a single event code
        # Choose experimental conditions: this example deals with "familiar" versus "unfamiliar" motor sequences
        'familiar': {'include': [3, 4, 30], 'exclude': [11, 12], 'merge': [3, 4], 'new_code': 1},
        'unfamiliar': {'include': [11, 12, 30], 'exclude': [3, 4], 'merge': [11, 12], 'new_code': 2},
    },
    'epoch_params': {
        'tmin': -6.5,
        'tmax': 0,
        'baseline': (-6.5, -5.5),
        'reject_criteria': dict(eeg=150e-6),
        'flat_criteria': dict(eeg=5e-6),
        'channels': ['F2', 'F4', 'F8', 'F1', 'F3', 'F7', 'Fz', 'FCz']
    },
    'tfr_params': {
        'freqs': np.arange(4, 14, 1),
        'n_cycles': 3,
        'use_fft': True,
        'n_jobs': 1,
    },
    'roi_definitions': {
        # Define ROIs as groups of channels (for averaging later)
        'LPF': ['F7', 'F3', 'F1'],
        'RPF': ['F2', 'F4', 'F8'],
        'CC' : ['Fz', 'FCz']
        # Add more ROIs as needed
    },
    'time_selection': {
        # Time windows for baseline vs activation window, adjust as necessary
        'baseline_window': (-6.5, -5.5),
        'activation_window': (-1.5, 0),
        'desired_times': [-6.5, -6.4, -6.3, -6.2, -6.1, -6.0, -5.9, -5.8, -5.7, -5.6, -5.5,
                          -1.5, -1.4, -1.3, -1.2, -1.1, -1.0, -0.9, -0.8, -0.7, -0.6, -0.5,
                          -0.4, -0.3, -0.2, -0.1, 0]
    },
    'output_paths': {
        # Change to your output path
        'export_dir': r'C:\Users\...'
    }

## Function definitions
Below we define functions for the different steps of the pipeline. Adjust as necessary:

loading, event creation, merging and re-labelling, trial segmentation, epoch creation, TFR computation, ERD/ERS transformation

In [ ]:
def load_raw_data(filepath):
    """Load raw data from a .fif file."""
    raw = mne.io.read_raw_fif(filepath, preload=True)
    return raw

def create_events(raw, event_id_map):
    """
    Create events from annotations using a provided event_id mapping.
    Returns events and a dictionary mapping.
    """
    events, event_dict = mne.events_from_annotations(raw, event_id=event_id_map)
    return events, event_dict

def merge_and_relabel_events(events, merge_def):
    """
    Given an events array, pick and merge events based on include/exclude criteria.
    This function uses mne.pick_events and mne.merge_events.
    """
    # Pick events that match "include" and exclude unwanted events.
    picked_events = mne.pick_events(events, include=merge_def['include'], exclude=merge_def['exclude'])
    # Merge the events in the merge list to new_code:
    merged_events = mne.merge_events(picked_events, merge_def['merge'], merge_def['new_code'], replace_events=True)
    return merged_events

def segment_trials(events, trial_ranges):
    """
    Segment the events array into trials based on provided ranges.
    `trial_ranges` should be a list of tuples: (start_index, end_index, new_label)
    """
    trials = np.copy(events)
    for start, end, new_label in trial_ranges:
        trials[start:end+1, 2] = new_label
    return trials

def create_epochs(raw, events, event_code, epoch_params, picks):
    """
    Create epochs from raw data given events and epoch parameters.
    """
    epochs = mne.Epochs(raw, events, event_id=event_code,
                        tmin=epoch_params['tmin'], tmax=epoch_params['tmax'],
                        baseline=epoch_params['baseline'], picks=picks,
                        reject=epoch_params['reject_criteria'], flat=epoch_params['flat_criteria'],
                        detrend=1, reject_by_annotation=True, preload=True)
    return epochs

def compute_tfr(epochs, tfr_params):
    """
    Compute time-frequency representations (Morlet wavelets) for given epochs.
    Returns power and inter-trial coherence (itc) if requested.
    """
    power, itc = tfr_morlet(epochs, freqs=tfr_params['freqs'],
                            n_cycles=tfr_params['n_cycles'],
                            use_fft=tfr_params['use_fft'],
                            return_itc=True,
                            n_jobs=tfr_params['n_jobs'])
    return power, itc

def process_power_to_ERDS(power, roi_defs, time_sel, export_label, participant, hand='Right', block='B1', condition='Familiar'):
    """
    Process the TFR power data:
      - Convert to a pandas DataFrame.
      - Limit time output to baseline and activation windows.
      - Average across channels for each ROI.
      - Smooth data by calculating a rolling mean.
      - Select only desired time values.
      - Compute baseline power and then calculate ERDS.
      - Reshape the DataFrame into a long format with ROI, time, and ERDS columns.

    Returns the processed DataFrame.
    """
    df = power.to_data_frame(time_format=None)
    # Add identifiers, change as needed
    df['Block'] = block
    df['Participant'] = participant
    df['Hand'] = hand
    df['Condition'] = condition

    # Step 1: Limit time output to baseline and activation windows
    df = df[(df['time'].between(*time_sel['baseline_window'])) | (df['time'].between(*time_sel['activation_window']))]

    # Step 2: Merge channels for each ROI and drop original channels
    for roi, channels in roi_defs.items():
        df[roi] = df[channels].mean(axis=1).astype(float)
        df.drop(columns=channels, inplace=True)

    # Step 3: Smooth data (here we use a rolling mean with window=50 and then repeat values)
    for roi in roi_defs.keys():
        mean_col = roi + '_mean'
        df[mean_col] = df[roi].rolling(window=50, min_periods=1).mean()
        # Expand repeated values
        df[mean_col] = np.repeat(df[mean_col].values[::50], 50)[:len(df)]
        df[roi] = df[mean_col]
        df.drop(columns=[mean_col], inplace=True)

    # Step 4: Remove redundant time values
    df = df[df['time'].isin(time_sel['desired_times'])]

    # Step 5: Compute baseline power per frequency (group by frequency)
    baseline_df = df[(df['time'] >= time_sel['baseline_window'][0]) & (df['time'] <= time_sel['baseline_window'][1])]
    baseline_power = baseline_df.groupby('freq')[list(roi_defs.keys())].mean().reset_index()
    # Rename baseline columns for merging
    baseline_power = baseline_power.rename(columns={roi: 'baseline_' + roi for roi in roi_defs.keys()})

    # Step 6: Merge and compute ERDS for each ROI
    merged = pd.merge(df, baseline_power, on='freq')
    for roi in roi_defs.keys():
        merged['ERDS_' + roi] = ((merged[roi] - merged['baseline_' + roi]) / merged['baseline_' + roi]) * 100
    # Drop extra columns
    merged.drop(columns=list(roi_defs.keys()) + ['baseline_' + roi for roi in roi_defs.keys()], inplace=True)

    # Step 7: Reshape DataFrame from wide to long format
    long_df = pd.melt(merged, id_vars=['time', 'freq', 'Block', 'Participant', 'Hand', 'Condition'],
                      var_name='ROI', value_name='ERDS')
    # Remove extra text (if any) from ROI names (e.g., remove 'ERDS_' prefix)
    long_df['ROI'] = long_df['ROI'].str.replace('ERDS_', '')

    # Optionally, drop baseline times if not needed in final dataset
    long_df = long_df[(long_df['time'] < time_sel['baseline_window'][0]) | (long_df['time'] > time_sel['baseline_window'][1])]

    # Export processed data to CSV
    export_path = os.path.join(params['output_paths']['export_dir'], f"{export_label}_{participant}.csv")
    long_df.to_csv(export_path, index=False)
    print(f"Exported processed data to {export_path}")

    return long_df

## Main pipeline execution

Here, we execute the pipeline for a given participant:

In [ ]:
def run_pipeline_for_participant(participant_label, filepath, params):
    """
    Run the full ERD/ERS pipeline for a single participant.
    """
    print(f"Processing participant {participant_label}")

    # 1. Load data
    raw = load_raw_data(filepath)

    # 2. Create events from annotations
    events, event_dict = create_events(raw, params['event_ids'])

    # 3. Create merged events for familiar and unfamiliar conditions
    familiar_events = merge_and_relabel_events(events, params['merge_events']['familiar'])
    unfamiliar_events = merge_and_relabel_events(events, params['merge_events']['unfamiliar'])

    # 4. (Optional) Create target events and segment trials if desired.
    # For simplicity, we show the basic event creation. You can add segmentation using segment_trials()
    # if you have pre-defined trial ranges.

    # 5. Define channel picks based on the provided channel list
    picks = mne.pick_channels(raw.info["ch_names"], params['epoch_params']['channels'])

    # 6. Create epochs for familiar and unfamiliar conditions
    epochs_fam = create_epochs(raw, familiar_events, event_code=params['merge_events']['familiar']['new_code'],
                               epoch_params=params['epoch_params'], picks=picks)
    epochs_unfam = create_epochs(raw, unfamiliar_events, event_code=params['merge_events']['unfamiliar']['new_code'],
                                 epoch_params=params['epoch_params'], picks=picks)

    # 7. Compute TFRs for each condition
    power_fam, itc_fam = compute_tfr(epochs_fam, params['tfr_params'])
    power_unfam, itc_unfam = compute_tfr(epochs_unfam, params['tfr_params'])

    # 8. Process power data into ERDS metrics
    fam_df = process_power_to_ERDS(power_fam, params['roi_definitions'],
                                   params['time_selection'],
                                   export_label="fam", participant=participant_label,
                                   hand='Right', block='B1', condition='Familiar')
    unfam_df = process_power_to_ERDS(power_unfam, params['roi_definitions'],
                                     params['time_selection'],
                                     export_label="unfam", participant=participant_label,
                                     hand='Right', block='B1', condition='Unfamiliar')

    # 9. (Optional) Combine the familiar and unfamiliar datasets for further analysis
    final_df = pd.concat([fam_df, unfam_df], ignore_index=True)
    export_path_final = os.path.join(params['output_paths']['export_dir'], f"final_{participant_label}.csv")
    final_df.to_csv(export_path_final, index=False)
    print(f"Exported final combined dataset to {export_path_final}")

    # 10. Plot events (for quick inspection)
    fig = mne.viz.plot_events(np.concatenate([familiar_events, unfamiliar_events], axis=0),
                              event_id={'familiar': params['merge_events']['familiar']['new_code'],
                                        'unfamiliar': params['merge_events']['unfamiliar']['new_code']},
                              sfreq=raw.info['sfreq'])
    plt.show()

    # 11. Return final DataFrame if needed
    return final_df

## Run the pipeline for multiple participants

For example, run for the first participant (P1). You can look over all keys in params['participant_files'].

In [ ]:
# Example: run for participant P1
participant_label = "P1"
filepath = params['participant_files'][participant_label]
final_dataset = run_pipeline_for_participant(participant_label, filepath, params)

## Next steps

- Review the exported CSV files
- Adjust any parameters as necessary (e.g., channel picks, epoch times, frequency ranges, etc.).
- Extend the pipeline (e.g., add trial segmentation or additional ROI definitions) as needed.
- Repeat for additional participants by looping over the file paths in `params['participant_files']`.

This should allow you to easily update and reuse the processing code for future datasets.
